In [1]:
import psycopg2
import wget
import pandas as pd
import sqlite3

In [2]:
# define user, dbname, password, and host (from ElephantSQL)

dbname = 'ceforqty'

user = 'ceforqty'

password = 'GYh0O6sCjsdPPgZXg4DRYzL3Y9LDC5gl'

host = 'salt.db.elephantsql.com'

In [3]:
# establish connection to elephantsql
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)

In [4]:
# create cursor
pg_curs = pg_conn.cursor()

In [5]:
# create dataframe
df = pd.read_csv('titanic.csv')
df.columns = ('survived', 'pclass', 'name', 'sex', 'age', 'sib_sp_aboard', 'par_ch_aboard', 'fare')
df['name'] = df['name'].str.replace("'", "")

In [6]:
df.tail()

,survived,pclass,name,sex,age,sib_sp_aboard,par_ch_aboard,fare
882,0,2,Rev. Juozas Montvila,male,27.0,0,0,13.00
883,1,1,Miss. Margaret Edith Graham,female,19.0,0,0,30.00
884,0,3,Miss. Catherine Helen Johnston,female,7.0,1,2,23.45
885,1,1,Mr. Karl Howell Behr,male,26.0,0,0,30.00
886,0,3,Mr. Patrick Dooley,male,32.0,0,0,7.75


In [7]:
# establish connection
t_conn = sqlite3.connect('Titanic.sqlite3')
df.to_sql('Titanic', con=t_conn)

ValueError: Table 'Titanic' already exists.

In [ ]:
# create cursor
t_curs = t_conn.cursor()

In [ ]:
# PRAGMA
t_curs.execute('PRAGMA table_info(Titanic);').fetchall()

In [ ]:
# create table
create_titanic_table = """
  CREATE TABLE titanic (
    index SERIAL PRIMARY KEY,
    survived INT,
    pclass INT,
    name TEXT,
    sex TEXT,
    age REAL,
    sib_sp_aboard INT,
    par_ch_aboard INT,
    Fare REAL
  );
"""

In [ ]:
# execute creation of table
pg_curs.execute(create_titanic_table)

In [ ]:
# We can query tables if we want to check
# This is a clever optional thing, showing postgresql internals
show_tables = """
SELECT
   *
FROM
   pg_catalog.pg_tables
WHERE
   schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
"""
pg_curs.execute(show_tables)
pg_curs.fetchall()

In [ ]:
# extract people list
people = t_curs.execute('SELECT * FROM Titanic;').fetchall()

In [ ]:
# How do we do this for all characters? Loops!
for person in people:
  insert_person = """
    INSERT INTO Titanic
    (survived, pclass, name, sex, age, sib_sp_aboard, par_ch_aboard, fare)
    VALUES """ + str(person[1:]) + ';'
# print(insert_person)
  pg_curs.execute(insert_person)

In [ ]:
pg_curs.close()
pg_conn.commit()